# Lab 01: Vanilla RNN - demo

In [5]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 153 sec ( 2.55 min)<br> 
* Time for 1 epoch on GPU : 8.4 sec w/ GeForce GTX 1080 Ti <br>

In [4]:
device= torch.device("cuda")
#device= torch.device("cpu")
print(device)

cuda


### Download Penn Tree Bank

The tensor train_data consists of 20 columns of 46,479 words.<br>
The tensor test_data consists of 20 columns of 4,121 words.

In [3]:
from utils import check_ptb_dataset_exists
data_path=check_ptb_dataset_exists()

train_data  =  torch.load(data_path+'ptb/train_data.pt')
test_data   =  torch.load(data_path+'ptb/test_data.pt')

print(  train_data.size()  )
print(  test_data.size()   )

torch.Size([46479, 20])
torch.Size([4121, 20])


### Some constants associated with the data set

In [4]:
bs = 20

vocab_size = 10000

### Make a recurrent net class

In [5]:
class three_layer_recurrent_net(nn.Module):

    def __init__(self, hidden_size):
        super(three_layer_recurrent_net, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.RNN(       hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1( word_seq )  
        h_seq , h_final     =   self.layer2( g_seq , h_init )
        score_seq           =   self.layer3( h_seq )
        
        return score_seq,  h_final 


### Build the net. Choose the hidden size to be 150. How many parameters in total?

In [6]:
hidden_size=150

net = three_layer_recurrent_net( hidden_size )

print(net)

utils.display_num_param(net)

three_layer_recurrent_net(
  (layer1): Embedding(10000, 150)
  (layer2): RNN(150, 150)
  (layer3): Linear(in_features=150, out_features=10000, bias=True)
)
There are 3055300 (3.06 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [7]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [8]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 1
* sequence length = 35

In [9]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

### Function to evaluate the network on the test set

In [10]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    
    h=h.to(device)

       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h  = net( minibatch_data, h )
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )
        

### Do 10 passes through the training set (100 passes would reach 135 on test set)

In [11]:
start=time.time()

for epoch in range(10):
    
    # keep the learning rate to 1 during the first 4 epochs, then divide by 1.1 at every epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # create a new optimizer and give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(1, bs, hidden_size)

    # send it to the gpu    
    h=h.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
             
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        h=h.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h  = net( minibatch_data, h )
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 



epoch= 0 	 time= 8.451735258102417 	 lr= 1 	 exp(loss)= 525.0974519777113
test: exp(loss) =  330.8869903548741

epoch= 1 	 time= 17.060683012008667 	 lr= 1 	 exp(loss)= 266.4897300199702
test: exp(loss) =  243.12823945558034

epoch= 2 	 time= 25.699626207351685 	 lr= 1 	 exp(loss)= 207.75759767275846
test: exp(loss) =  211.25912074894677

epoch= 3 	 time= 34.35404300689697 	 lr= 1 	 exp(loss)= 177.64197124571623
test: exp(loss) =  193.25772664137736

epoch= 4 	 time= 43.01669692993164 	 lr= 0.9090909090909091 	 exp(loss)= 156.6999821913823
test: exp(loss) =  182.01967164933876

epoch= 5 	 time= 51.71618413925171 	 lr= 0.8264462809917354 	 exp(loss)= 142.2161103899622
test: exp(loss) =  172.8387773264008

epoch= 6 	 time= 60.4546685218811 	 lr= 0.7513148009015777 	 exp(loss)= 131.75135493279933
test: exp(loss) =  167.60669838877308

epoch= 7 	 time= 69.11849451065063 	 lr= 0.6830134553650705 	 exp(loss)= 123.44600337358432
test: exp(loss) =  161.1225279934804

epoch= 8 	 time= 77.76724

### Choose one sentence (taken from the test set)

In [12]:
sentence1 = "some analysts expect oil prices to remain relatively"

sentence2 = "over the next days and weeks they say investors should look for stocks to"

sentence3 = "prices averaging roughly $ N a barrel higher in the third"

sentence4 = "i think my line has been very consistent mrs. hills said at a news"

sentence5 = "this appears particularly true at gm which had strong sales in"

# or make your own sentence.  No capital letter or punctuation allowed. Each word must be in the allowed vocabulary.
sentence6= "he was very"

# SELECT THE SENTENCE HERE
mysentence = sentence1

### Convert the sentence into a vector, then send to GPU

In [13]:
minibatch_data=utils.sentence2vector(mysentence)
      
minibatch_data=minibatch_data.to(device)

print(minibatch_data)

tensor([[ 307],
        [1140],
        [ 334],
        [1486],
        [1786],
        [  64],
        [ 719],
        [ 377]], device='cuda:0')


### Set the initial hidden state to zero, then run the RNN.

In [14]:
h = torch.zeros(1, 1, hidden_size)
h=h.to(device)

scores , h = net( minibatch_data , h )

### Display the network prediction for the next word

In [15]:
print(mysentence, '... \n')

utils.show_next_word(scores)

some analysts expect oil prices to remain relatively ... 

6.4%	 much
5.4%	 <unk>
3.0%	 low
2.0%	 N
1.8%	 little
1.7%	 high
1.5%	 more
1.5%	 share
1.3%	 active
1.3%	 the
1.2%	 good
1.2%	 higher
1.2%	 big
0.9%	 strong
0.9%	 modest
0.7%	 well
0.7%	 in
0.7%	 positive
0.7%	 part
0.6%	 lower
0.6%	 large
0.6%	 flat
0.6%	 long
0.5%	 less
0.5%	 a
0.5%	 close
0.5%	 expensive
0.5%	 very
0.5%	 important
0.5%	 far
